In [14]:
import numpy as np
import tensorflow as tf
import tensorflow.keras.datasets.mnist as mnist
from datetime import datetime

In [2]:
data_path = "/Users/bryan/Documents/sandbox/data/mnist.npz"
debug = True

validation_ratio = 0.2
lr = 3e-4
weight_decay = 0.1
epochs = 100

In [3]:
def get_validation_split_indicies(data_length, val_ratio=0.2):
    validation_amount = int(data_length * val_ratio)
    p = np.random.choice(range(data_length), data_length, replace=False)
    return p[:-validation_amount], p[-validation_amount:]

In [4]:
# Get data from MNIST
(train_x, train_y), (test_x, test_y) = mnist.load_data(path=data_path)
train_idx, valid_idx = get_validation_split_indicies(len(train_x), validation_ratio)

validation_x = train_x[valid_idx]
validation_y = train_y[valid_idx]

train_x = train_x[train_idx]
train_y = train_y[train_idx]

In [5]:
height, width = train_x.shape[1:]
hidden_output_size = 100
output_size = 10

In [34]:
def fully_connected(x, output_size, activation=None, name="fully_connected", weight_init=None, bias_init=None):
    with tf.variable_scope(name):
        w_init = weight_init if weight_init else tf.contrib.layers.xavier_initializer()
        b_init = bias_init if bias_init else tf.zeros_initializer()

        W = tf.get_variable("weight", shape=[x.shape[1], output_size], initializer=w_init, trainable=True)
        b = tf.get_variable("bias", shape=[output_size, ], initializer=b_init, trainable=True)

        y = tf.matmul(x, W) + b

        if activation:
            return activation(y), W, b

        return y, W, b

In [38]:
tf.reset_default_graph()

with tf.name_scope("placeholders"):
    x = tf.placeholder(tf.float32, (None, height, width))
    y = tf.placeholder(tf.int64, (None,))

with tf.name_scope("flatten"):
    flattened_x = tf.reshape(x, (-1, np.product(x.shape[1:])))
    
with tf.name_scope("hidden"):
    sigma, _, _ = fully_connected(flattened_x, hidden_output_size, tf.nn.relu)
    
with tf.name_scope("output"):
    initializer = tf.contrib.layers.xavier_initializer()
    W = tf.get_variable(name="output_weight", shape=[hidden_output_size, output_size], 
                        initializer=tf.contrib.layers.xavier_initializer(), trainable=True)
    b = tf.get_variable(name="output_bias", shape=[output_size,], initializer=tf.zeros_initializer(), 
                        trainable=True)
    logits = tf.matmul(sigma, W) + b

with tf.name_scope("take_max"):
    predictions = tf.argmax(logits, 1)

In [36]:
loss = tf.losses.sparse_softmax_cross_entropy(y, logits)
accuracy = tf.reduce_mean(tf.cast(tf.equal(y, predictions), tf.float32))
trainer = tf.train.AdamOptimizer(lr).minimize(loss)

with tf.name_scope("summaries"):
    tf.summary.scalar("loss", loss)
    tf.summary.scalar("accuracy", accuracy)
    merged = tf.summary.merge_all()

In [37]:
with tf.Session() as sess:
    start_time = datetime.now().strftime("%m-%d-%Y_%H-%M-%S")
    train_writer = tf.summary.FileWriter(logdir="./runs/{}_train".format(start_time))
    validation_writer = tf.summary.FileWriter(logdir="./runs/{}_validation".format(start_time))
    sess.run(tf.group(tf.global_variables_initializer(), tf.local_variables_initializer()))
    for epoch in range(epochs):
        _, train_summary, train_loss, train_accuracy = sess.run([trainer, merged, loss, accuracy], feed_dict={x: train_x, y: train_y})
        train_writer.add_summary(train_summary, epoch)
        
        validation_summary, validation_loss, validation_accuracy = sess.run([merged, loss, accuracy], feed_dict={x: validation_x, y: validation_y})
        validation_writer.add_summary(validation_summary, epoch)       

        if (epoch + 1) % 10 == 0:
            print("Epoch: {} ==================".format(epoch + 1))
            print("Training - Loss: {}, Accuracy: {}".format(train_loss, train_accuracy))
            print("Validation - Loss: {}, Accuracy: {}".format(validation_loss, validation_accuracy))
        

Epoch: 10 ==================
Training - Loss: 62.41252136230469, Accuracy: 0.2057916671037674
Validation - Loss: 58.17912673950195, Accuracy: 0.22550000250339508
Epoch: 20 ==================
Training - Loss: 31.059356689453125, Accuracy: 0.42664584517478943
Validation - Loss: 29.317852020263672, Accuracy: 0.45350000262260437
Epoch: 30 ==================
Training - Loss: 17.022499084472656, Accuracy: 0.6190208196640015
Validation - Loss: 16.820898056030273, Accuracy: 0.6272500157356262
Epoch: 40 ==================
Training - Loss: 12.452691078186035, Accuracy: 0.7024999856948853
Validation - Loss: 12.568055152893066, Accuracy: 0.6977499723434448
Epoch: 50 ==================
Training - Loss: 9.977433204650879, Accuracy: 0.7503958344459534
Validation - Loss: 10.21493911743164, Accuracy: 0.7463333606719971
Epoch: 60 ==================
Training - Loss: 8.398648262023926, Accuracy: 0.7800208330154419
Validation - Loss: 8.750054359436035, Accuracy: 0.7735000252723694
Epoch: 70 ===============

1560028203.773414